In [154]:
import pandas as pd
import dimod

## QUBO encoder

### Data description
Lorem ipsum ...
### Read spot file

In [283]:
# create empty photo requests dataframe
photo_req_df = pd.DataFrame({'id':      pd.Series(dtype='int'),
                             'value':   pd.Series(dtype='int'),
                             'mono':    pd.Series(dtype='bool'),
                             'options': pd.Series(dtype='object')})

In [284]:
# process photo requests
def request_from_line(line):
    # split line
    l = line.split()

    # get features
    id = int(l[0])
    value = int(l[1])
    mono = False if (l[2] == '1' and int(l[3])>4) else True
    options = [int(o) for o in l[3::2]]

    return list([id, value, mono, options])

In [285]:
# create empty constraints dataframe
constraints_df = pd.DataFrame({'ids':            pd.Series(dtype='object'),
                               'restrictions':   pd.Series(dtype='object')})

In [286]:
# process constraints
def costraint_from_line(line):
    # split line
    l = line.split()

    # get features
    n = int(l[0])
    ids = [int(i) for i in l[1:n+1]]
    restrictions = [[int(l[j]) for j in range(i,i+n)] for i in range(n+1, len(l), n)]

    return list([ids, restrictions])

In [287]:
# read test file
data_dir  = 'data/'
file_name = '1405.spot'
with open(data_dir+file_name) as f:
    lines = f.readlines()

# get ids and reqs lengths
lens = [int(l.split()[0]) for l in lines if len(l.split())==1]

# create photo requests dataframe
for i in range(1,lens[0]+1):
    photo_req_df.loc[i-1] = request_from_line(lines[i])

# create constraints dataframe
for i in range(lens[0]+2, lens[0]+lens[1]+2):
    constraints_df.loc[i-(lens[0]+2)] = costraint_from_line(lines[i])

In [304]:
photo_req_df

,id,value,mono,options
0,0,1000,True,[2]
1,1,1000,True,[2]
2,2,1000,True,[2]
3,3,1000,True,[2]
4,4,1000,True,[2]
...,...,...,...,...
850,850,1,True,"[1, 2, 3]"
851,851,1,True,"[1, 2, 3]"
852,852,1,True,"[1, 2, 3]"
853,853,1,True,"[1, 2, 3]"


In [61]:
constraints_df

,ids,restrictions
0,"[1, 0]","[[3, 3], [2, 2], [1, 1]]"
1,"[2, 0]","[[3, 3], [2, 2], [1, 1]]"
2,"[3, 0]","[[3, 3], [2, 2], [1, 1]]"
3,"[5, 4]","[[13, 13]]"
4,"[5, 6]","[[13, 13]]"
5,"[2, 1]","[[3, 3], [2, 2], [1, 1]]"
6,"[3, 1]","[[3, 3], [2, 2], [1, 1]]"


### QUBO formulation
QUBO stays for Quadratic Uncostrained _Binary_ Optimization. First of all, we should translate our problem in terms of binary variables. We will consider two possibilities:

__standard__
- _mono_: We can take (1) or not take (0) the photo with each of the three cameras. Three binary variables are needed:
$$
(x_{i0},\, x_{i1},\, x_{i2})
$$
- _stereo_: The stereo photo can only be taken with the cameras $1$ and $3$, so we have only one binary variable:
$$
x_i
$$

Note that, in the case of a mono photo, we have a total of 8 different instantiations of the binary variables when only 3 are feasible ($001,\, 010,\, 100$). We can think about more efficient encodings.

__dense__:
- _mono_: Just two variables could be sufficient:

<center>

|          | $x_{i0}$ | $x_{i1}$ |
|----------|----------|----------|
| no photo | 0        | 0        |
| camera 1 | 0        | 1        |
| camera 2 | 1        | 0        |
| camera 3 | 1        | 1        |

</center>

- _stereo_: As in the previous encoding we consider one variable
$$
x_i
$$

In the latter case we use one less variable with respect to the former, it seems convenient. Despite that, with the _dense_ encoding the costraint formulation requires an auxiliary variable so it is important to carefully understand if there is an actual advantage.

In [269]:
# create empty qubo dataframe
qubo_df = pd.DataFrame({'rank':    pd.Series(dtype='int'),
                        'coeff':   pd.Series(dtype='int'),
                        'indexes': pd.Series(dtype='object')})

In [270]:
# formulate qubo istance from photo request
def qubo_from_request(request, option):
    # get features
    id = request[0]
    value = request[1]
    mono = request[2]
    camera = request[3][option]

    # get qubo
    rank = 1
    coeff = -value # minimize qubo -> maximize value
    indexes = [[id, camera]] # if mono else [id]
    qubo = list([rank, coeff, indexes])

    return qubo

In [271]:
 # formulate qubo instance from constraint
def qubo_from_constraint(constraint, option, coeff):
    # get features
    ids = constraint[0]
    restriction = constraint[1][option]

    # get qubo
    rank = len(ids)
    coeff = coeff
    indexes = [[id, camera] for id, camera in zip(ids, restriction)]
    qubo = list([rank, coeff, indexes])

    return qubo

In [272]:
k = 0

# populate qubo dataframe from photo requests
for i in range(len(photo_req_df)):
    l = len(photo_req_df.loc[i]['options'])
    for j in range(l):
        qubo_df.loc[k+j] = qubo_from_request(photo_req_df.loc[i], j)
    k = k + l

# populate qubo dataframe from constraints
m = -1.1*min(qubo_df['coeff'])                                                                           # penalties coefficient
for i in range(len(constraints_df)):
    l = len(constraints_df.loc[i]['restrictions'])
    for j in range(l):
        qubo_df.loc[k+j] = qubo_from_constraint(constraints_df.loc[i], j, m)
    k = k + l

In [275]:
qubo_df.head()

,rank,coeff,indexes
0,1,-1.0,"[[0, 1]]"
1,1,-1.0,"[[0, 2]]"
2,1,-1.0,"[[0, 3]]"
3,1,-1.0,"[[1, 1]]"
4,1,-1.0,"[[1, 2]]"


In [276]:
"""# group qubo dataframe by indexes TODO: check if it is necessary ( it seems not )
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes

# group by operation needs string object
qubo_df['gb_indexes'] = qubo_df['indexes'].apply(preprocess_indexes, join=True)
qubo_df_ = qubo_df.groupby(['gb_indexes'], as_index=False, sort=False).agg({'rank': 'first', 'coeff': 'sum', 'indexes': 'first'})#.drop(['gb_indexes'], axis=1)
# recover process indexes for qubo indexing
#qubo_df_['proc_indexes'] = qubo_df_['indexes'].apply(preprocess_indexes, join=False)"""
def preprocess_indexes(indexes, join=False):
    # sort if more than one element
    if len(indexes)>1:
        indexes = sorted(indexes, key=lambda x: x[0]+x[1]/10) # hardcoded solution to consider both sublists elements

    # produce string output
    proc_indexes = [str(i[0])+str(i[1]) for i in indexes ]
    return ''.join(proc_indexes) if join else proc_indexes
    
# produce hash column for dictionary indexing
qubo_df['keys'] = qubo_df['indexes'].apply(preprocess_indexes, join=False)

In [277]:
# produce 1 to one dict between keys of the dataframe and indexes of the qubo matrix
key_to_qubo_dict = {}
for i in range(len(qubo_df)):
    # check if index is a single key
    if len(qubo_df.loc[i]['keys'])==1:
        key_to_qubo_dict[qubo_df.loc[i]['keys'][0]] = i

In [278]:
# populate qubo matrix dictionary
qubo = {}
for i in range(len(qubo_df)):
    # check if index is a single key
    keys = qubo_df.loc[i]['keys']
    if len(keys)==1:
        idx = key_to_qubo_dict[keys[0]]
        # check if index is already in dictionary
        qubo[(idx, idx)] = qubo[(idx, idx)]+qubo_df.loc[i]['coeff'] if (idx, idx) in qubo else qubo_df.loc[i]['coeff']
    else:
        # assume quadratic term, higher order will be treated before 
        idx_0 = key_to_qubo_dict[keys[0]]
        idx_1 = key_to_qubo_dict[keys[1]]
        # check if index is already in dictionary
        qubo[(idx_0, idx_1)] = qubo[(idx_0, idx_1)]+qubo_df.loc[i]['coeff'] if (idx_0, idx_1) in qubo else qubo_df.loc[i]['coeff']

In [279]:
# print qubo matrix
def print_qubo(qubo, key_to_qubo_dict):
    for i in range(len(key_to_qubo_dict)):
        for j in range(len(key_to_qubo_dict)):
            print(qubo[(i, j)], end=' ') if (i,j) in qubo else print(0, end=' ')
        print()

print_qubo(qubo, key_to_qubo_dict)

-1.0 0 0 2.2 0 0 2.2 0 0 2.2 0 0 0 0 0 0 
0 -1.0 0 0 2.2 0 0 2.2 0 0 2.2 0 0 0 0 0 
0 0 -1.0 0 0 2.2 0 0 2.2 0 0 2.2 0 0 0 0 
0 0 0 -1.0 0 0 2.2 0 0 2.2 0 0 0 0 0 0 
0 0 0 0 -1.0 0 0 2.2 0 0 2.2 0 0 0 0 0 
0 0 0 0 0 -1.0 0 0 2.2 0 0 2.2 0 0 0 0 
0 0 0 0 0 0 -1.0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 -1.0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 -1.0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 -1.0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 -1.0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 -1.0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 -2.0 2.2 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 2.2 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2.0 


### Exact solver

In [280]:
# encode the dataframe into a qubo object
bqm = dimod.BQM.from_qubo(qubo)
# exact solver
sampler_exact = dimod.ExactSolver()
sampleset = sampler_exact.sample(bqm)

In [281]:
# save results in a dataframe
results_df = sampleset.to_pandas_dataframe()
# recover original indexes
qubo_to_key_dict = {v: k for k, v in key_to_qubo_dict.items()}
results_df.columns = [qubo_to_key_dict[q] for q in results_df.columns[:-2]]+['energy', 'num_occurrences']
# sort by energy
results_df = results_df.sort_values(by=['energy'], ascending=True)

In [282]:
results_df

,01,02,03,11,12,13,21,22,23,31,32,33,413,513,613,713,energy,num_occurrences
38271,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,-12.0,1
38654,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,1,-11.0,1
39800,0,0,1,0,0,0,1,1,0,1,1,0,1,0,1,1,-11.0,1
39744,0,0,0,0,0,1,1,1,0,1,1,0,1,0,1,1,-11.0,1
37472,0,0,0,0,1,0,1,0,1,1,0,1,1,0,1,1,-11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13653,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,19.0,1
19114,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,19.2,1
10922,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,19.2,1
21845,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,19.4,1
